# Welcome to drugforge

Welcome to the drugforge tutorial series! 

This notebook will run you through some of the base level abstractions used in our workflows and get you comfortable with the style of the package! 

## Making your first Ligand

We aim to provide high-level abstractions that allow conceptual operations on common objects in drug discovery without worrying about implementation details.

Nothing could be more fundamental to drug discovery than a ligand so lets start there! `drugforge` has a `Ligand` schema that acts as a metadata rich serializable wrapper around a small molecule (backed by an SDF string).  This is essential for allowing 

In [ ]:
from drugforge.data.schema.ligand import Ligand

# make a ligand from a SMILES string 

lig = Ligand.from_smiles("CC(Cc1ccc(cc1)C(C(=O)O)C)C", compound_name="ibuprofen") # compound name is mandatory 

In [ ]:
# we can compute common properties of our ligand
print(lig.inchi)
print(lig.inchikey)
print(lig.num_poses)
print(lig.smiles)

In [ ]:
# our representation is fully serialisable as JSON, backed by storing an SDF file
lig.json()

In [ ]:
# serialize to JSON
lig.to_json_file("my_ligand.json")

In [ ]:
# deserialize 
lig2 = Ligand.from_json_file("my_ligand.json")
# check for equality
lig == lig2

In [ ]:
# you can save it as an SDF file 
lig.to_sdf("my_sdf.sdf")

These abstractions enable remote transmission of ligands, easy metadata tracking and simple equality testing between small molecules. All of our workflows make ample use of these abstractions to avoid extensive metadata interrogation at each step and allow easy flow through of identifiers. 

In [ ]:
# we can also easily make OpenEye molecules from ligands to work with OpenEye components. 
oemol = lig.to_oemol()
oemol

In [ ]:
# we can also easily make RDKit molecules from ligands 
rdkit_mol = lig.to_rdkit()
rdkit_mol

These translations allow easy use with chemoinformatics,  structure based drug design toolkits and molecular simulation engines of all kinds.  

## Making your first Target

Most drug discovery campaigns need a target! So how does `drugforge` handle these? A `Target` is a metadata rich serializable  wrapper around a PDB file in much the same way as a `Ligand`. 

For this example we will use an ASAP target, the SARS-CoV-2 nsp3 Mac1 macrodomain that  removes ADP ribose from viral and host cell proteins. The removal of this post-translational modification reduces the inflammatory and antiviral responses to infection — facilitating replication (see [here](https://www.mdpi.com/2076-0817/11/1/94) for review).

See [SARS-CoV-2 nsp3 Mac1 targeting opportunity](https://asapdiscovery.notion.site/Targeting-Opportunity-SARS-CoV-2-nsp3-Mac1-macrodomain-47af24638b994e8ba786303ec743926e) for more information on Mac1. 


**NOTE: A target is designed to wrap only the protein component of a PDB file.** To work with a protein-ligand complex, you should use a `Complex` object (see later). Making a `Target` will automatically remove the small molecule components from a PDB file. 


In [ ]:
# first lets grab a file from the `asapdiscovery` test suite
from drugforge.data.testing.test_resources import fetch_test_file

In [ ]:
from drugforge.data.schema.target import Target

In [ ]:
protein = fetch_test_file("SARS2_Mac1A-A1013.pdb")
print(type(protein)) # its a path to a real file

In [ ]:
mac1_target = Target.from_pdb(protein, target_name="Mac1A")

In [ ]:
# serialize to JSON
mac1_target.to_json_file("target.json")

In [ ]:
# deserialize from JSON
t2 = Target.from_json_file("target.json")
t2 == mac1_target

In [ ]:
# also to a PDB file, only protein components included
mac1_target.to_pdb("my_pdb.pdb")

## Making your first Complex

We have looked at `Targets` and `Ligands` now what about combining them? A complex is just that, a combination of a ligand and target object for easy handling of both small molecule and protein elements


In [ ]:
from drugforge.data.schema.complex import Complex

In [ ]:
complx = Complex.from_pdb(protein, target_kwargs={"target_name": "Mac1A"}, ligand_kwargs={"compound_name": "A1013"})

In [ ]:
complx.ligand

In [ ]:
complx.target

In [ ]:
# can be serialized as one file with JSON 
complx.to_json_file("my_complex.json")

In [ ]:
c2 = Complex.from_json_file("my_complex.json")

In [ ]:
c2 == complx

In [ ]:
# you can make a combined OpenEye molecule easily
complx.to_combined_oemol()

In [ ]:
# or save as a PDB file, protein and ligand included
complx.to_pdb("my_complex.pdb")

## Summary

Hopefully this has given you a nice introduction to the base level abstractions used by the `asapdiscovery` repo. Continue on to the next tutorials for more fun stuff. 